#Transformer Chatbot with Cornell data

## Data loading and preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/cornell_movie_dialogs_corpus.zip" -d "/content/cornell_movie"

Archive:  /content/drive/MyDrive/cornell_movie_dialogs_corpus.zip
   creating: /content/cornell_movie/cornell movie-dialogs corpus/
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/.DS_Store  
   creating: /content/cornell_movie/__MACOSX/
   creating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/
  inflating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/chameleons.pdf  
  inflating: /content/cornell_movie/__MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_lines.txt  
  inflating: /content/cornell_movie/cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: /content/cornell_movie/cornell

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import re
import time
import os
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [ ]:
import codecs

with codecs.open("/content/cornell_movie/cornell movie-dialogs corpus/movie_lines.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    conversations = []
    for line in lines:
        data = line.split(" +++$+++ ")
        conversations.append(data)

In [ ]:
with codecs.open("/content/drive/MyDrive/short_questions.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    short_questions = []
    for line in lines:
        data = line.split("\n")[0]
        short_questions.append(data)

In [ ]:
short_questions = short_questions[:-1]
len(short_questions)

113507

In [ ]:
with codecs.open("/content/drive/MyDrive/short_answers.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    short_answers = []
    for line in lines:
        data = line.split("\n")[0]
        short_answers.append(data)

In [ ]:
import re
def clean_text(text):
    # Clean text by removing unnecessary characters and altering the format of words

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [ ]:
#choosing number of samples
num_samples = 30000  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]

In [ ]:
with codecs.open("/content/drive/MyDrive/faq_ques.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    faq_questions = []
    for line in lines:
        data = line.split("\n")[0]
        faq_questions.append(data)

In [ ]:
with codecs.open("/content/drive/MyDrive/faq_ans.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    faq_answers = []
    for line in lines:
        data = line.split("\n")[0]
        faq_answers.append(data)

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    short_questions + short_answers + faq_questions + faq_answers, target_vocab_size=2000)

# Define start and end token to indicate the start and end of a sentence
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# Vocabulary size plus start and end token
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
print(tokenizer.encode(short_questions[20]))

[423, 80, 37, 964, 51, 66, 24, 7, 329, 761]


In [ ]:
MAX_LENGTH = 16

In [ ]:
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # tokenize sentence
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
    # check tokenized sentence max length
    if len(sentence1) <=MAX_LENGTH  and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # pad tokenized sentences
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs


In [ ]:
questions, answers = tokenize_and_filter(short_questions, short_answers)

In [ ]:
print(VOCAB_SIZE, len(questions), len(answers))

1999 14528 14528


In [ ]:
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(1)

## Transformer Model

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :] 

In [ ]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
  """Calculate the attention weights. """
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # scale matmul_qk
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask to zero out padding tokens
  if mask is not None:
    logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(attention_weights, value)

  return output

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # concatenation of heads
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # final linear layer
    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)
  for i in range(int(num_layers)):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(int(num_layers)):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  # mask the future tokens for decoder inputs at the 1st attention block
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  # mask the encoder outputs for the 2nd attention block
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
tf.keras.backend.clear_session()

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

In [ ]:
model = transformer(
      vocab_size=VOCAB_SIZE,
      num_layers=NUM_LAYERS,
      units=UNITS,
      d_model=D_MODEL,
      num_heads=NUM_HEADS,
      dropout=DROPOUT)
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    1565952     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [ ]:
model.fit(dataset, epochs=40)

Epoch 1/40
908/908 [==============================] - 36s 30ms/step - loss: 3.1789 - accuracy: 0.0891
Epoch 2/40
908/908 [==============================] - 27s 30ms/step - loss: 2.5927 - accuracy: 0.1255
Epoch 3/40
908/908 [==============================] - 27s 30ms/step - loss: 2.3974 - accuracy: 0.1391
Epoch 4/40
908/908 [==============================] - 27s 30ms/step - loss: 2.2794 - accuracy: 0.1474
Epoch 5/40
908/908 [==============================] - 27s 30ms/step - loss: 2.1965 - accuracy: 0.1531
Epoch 6/40
908/908 [==============================] - 27s 30ms/step - loss: 2.0859 - accuracy: 0.1609
Epoch 7/40
908/908 [==============================] - 27s 30ms/step - loss: 1.9740 - accuracy: 0.1712
Epoch 8/40
908/908 [==============================] - 27s 30ms/step - loss: 1.8692 - accuracy: 0.1805
Epoch 9/40
908/908 [==============================] - 27s 30ms/step - loss: 1.7709 - accuracy: 0.1915
Epoch 10/40
908/908 [==============================] - 27s 30ms/step - loss: 1.680

In [ ]:
model.save_weights('transformer_2.0_40epochs.h5')

#### Evaluation and Inference

In [ ]:
def evaluate(sentence, model):
  sentence = clean_text(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [ ]:
def predict(sentence,model):
  prediction = evaluate(sentence,model)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
loaded_model = transformer(
      vocab_size=VOCAB_SIZE,
      num_layers=NUM_LAYERS,
      units=UNITS,
      d_model=D_MODEL,  
      num_heads=NUM_HEADS,
      dropout=DROPOUT)

#import h5py
#with h5py.File('saved_weights.h5', 'w') as f:
loaded_model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
loaded_model.load_weights('transformer_2.0_40epochs.h5')

In [ ]:
output = predict('Hello', loaded_model)

Input: Hello
Output: good day


In [ ]:
sentence = 'Hello there'
for _ in range(5):
  sentence = predict(sentence, loaded_model)
  print('')

Input: Hello there
Output: hello hello maria

Input: hello hello maria
Output: it is a calder

Input: it is a calder
Output: you are not a bitch

Input: you are not a bitch
Output: i am not sure that he is ffing at all

Input: i am not sure that he is ffing at all
Output: you are a moron you know that



## Fine Tuning

In [ ]:
faqs, ans = tokenize_and_filter(faq_questions, faq_answers)

In [ ]:
dataset2 = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': faqs,
        'dec_inputs': ans[:, :-1]
    },
    {
        'outputs': ans[:, 1:]
    },
))

dataset2 = dataset2.cache()
dataset2 = dataset2.shuffle(5)
dataset2 = dataset2.batch(1)
dataset2 = dataset2.prefetch(1)

In [ ]:
loaded_model.fit(dataset2, epochs=100)

Epoch 1/100
8/8 [==============================] - 5s 28ms/step - loss: 4.1442 - accuracy: 0.1333
Epoch 2/100
8/8 [==============================] - 0s 29ms/step - loss: 1.3056 - accuracy: 0.4333
Epoch 3/100
8/8 [==============================] - 0s 27ms/step - loss: 0.5043 - accuracy: 0.5500
Epoch 4/100
8/8 [==============================] - 0s 29ms/step - loss: 0.3172 - accuracy: 0.6000
Epoch 5/100
8/8 [==============================] - 0s 28ms/step - loss: 0.2198 - accuracy: 0.5833
Epoch 6/100
8/8 [==============================] - 0s 27ms/step - loss: 0.1691 - accuracy: 0.6167
Epoch 7/100
8/8 [==============================] - 0s 27ms/step - loss: 0.1362 - accuracy: 0.6083
Epoch 8/100
8/8 [==============================] - 0s 28ms/step - loss: 0.1296 - accuracy: 0.6250
Epoch 9/100
8/8 [==============================] - 0s 27ms/step - loss: 0.0704 - accuracy: 0.6333
Epoch 10/100
8/8 [==============================] - 0s 27ms/step - loss: 0.0504 - accuracy: 0.6333
Epoch 11/100
8/8 [=

In [ ]:
model.save_weights('fine_transformer_2.0_40_100epochs.h5')

In [ ]:
fine_loaded_model = transformer(
      vocab_size=VOCAB_SIZE,
      num_layers=NUM_LAYERS,
      units=UNITS,
      d_model=D_MODEL,  
      num_heads=NUM_HEADS,
      dropout=DROPOUT)

#import h5py
#with h5py.File('saved_weights.h5', 'w') as f:
fine_loaded_model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
fine_loaded_model.load_weights('fine_transformer_2.0_40_100epochs.h5')

In [ ]:
output = predict('my homework partner is not responding', fine_loaded_model)

Input: my homework partner is not responding
Output: please post a private message on edstem


In [ ]:
output = predict('is presentation a group activity', fine_loaded_model)

Input: is presentation a group activity
Output: recording links will be posted on edstem


In [ ]:
output = predict('i have a different approach to the exercise why is the autograder still failing me', fine_loaded_model)

Input: i have a different approach to the exercise why is the autograder still failing me
Output: recording links will be posted on edstem


#### Evaluation on test data

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
test.head()

,Questions
0,Will the pre-class session be recorded?
1,What is the deadline for quiz submission?
2,What is the deadline for exercise submission?
3,How many hours do I need to complete this course?
4,Who will grade the exercise?


In [ ]:
test.Questions.values[0]

'Will the pre-class session be recorded?'

In [ ]:
pred_test = []
for q in test.Questions.values:
  pred_test.append(predict(q, fine_loaded_model))

In [ ]:
predictions_test_data = pd.DataFrame()
predictions_test_data['Questions'] = test.Questions.values
predictions_test_data['Predicted_answers'] = pred_test

In [ ]:
predictions_test_data

,Questions,Predicted_answers
0,Will the pre-class session be recorded?,yes all sessions will be recorded
1,What is the deadline for quiz submission?,5 minutes
2,What is the deadline for exercise submission?,5 minutes
3,How many hours do I need to complete this course?,yes 4 of you together
4,Who will grade the exercise?,please post a private message on edstem
5,Why is the auto-grader failing me?,please post a private message on edstem
6,Do I do the exercises individually?,yes 4 of you together
7,Is the lab compulsory?,recording links will be posted on edstem
8,Will the sessions be recorded?,yes all sessions will be recorded
9,Can I have access to the recorded videos?,recording links will be posted on edstem


In [ ]:
predictions_test_data.to_csv('/content/drive/MyDrive/test_data_predictions_transformer_Cornell.csv',index=False)